# Structure Learning in Bayesian Networks

In this notebook, we show a few examples of Causal Discovery or Structure Learning in pgmpy. pgmpy currently has the following algorithm for causal discovery:

1. **PC**: Has $3$ variants original, stable, and parallel. PC is a constraint-based algorithm that utilizes Conditional Independence tests to construct the model.
2. **Hill-Climb Search**: Hill-Climb Search is a greedy optimization-based algorithm that makes iterative local changes to the model structure such that it improves the overall score of the model.
3. **Greedy Equivalence Search (GES)**: Another score-based method that makes greedy modifications to the model to improve its score iteratively.
4. **ExpertInLoop**: An iterative algorithm that combines Conditional Independence testing with expert knowledge. The user or an LLM can act as the expert.
5. **Exhaustive Search**: Exhaustive search iterates over all possible network structures on the given variables to find the most optimal one. As it tries to enumerate all possible network structures, it is intractable when the number of variables in the data is large.

The following Conditional Independence Tests are available to use with PC algorithm.
1. **Discrete Data**: When all variables are discrete/categorical.
    1. **Chi-square test**: `ci_test="chi_square"`
    2. **G-squared**: `ci_test="g_sq"`
    3. **Log-likelihood**: Is equivalent to G-squared test. `ci_test="log_likelihood`
2. **Continuous Data**: When all variables are continuous/numerical.
    1. **Partial Correlation**: `ci_test="pearsonr"`
3. **Mixed Data**: When there is a mix of categorical and continuous variables.
    1. **Pillai**: `ci_test="pillai"`

For Hill-Climb, Exhausitive Search, and GES the following scoring methods can be used:
1. **Discrete Data**: When all variables are discrete/categorical. 
    1. **BIC Score**: `scoring_method="bic-d"`
    2. **AIC Score**: `scoring_method="aic-d"`
    3. **K2 Score**: `scoring_method="k2"`
    4. **BDeU Score**: `scoring_method="bdeu"`
    5. **BDs Score**: `scoring_method="bds"`
2. **Continuous Data**: When all variables are continuous/numerical.
    1. **Log-Likelihood**: `scoring_method="ll-g"`
    2. **AIC**: `scoring_method="aic-g"`
    3. **BIC**: `scoring_method="bic-g"`
3. **Mixed Data**: When there is a mix of discrete and continuous variables.
    1. **AIC**: `scoring_method="aic-cg"`
    2. **BIC**: `scoring_method="bic-cg"`

## 0. Simulate some sample datasets

In [23]:
import sys
import importlib

# Forcer le rechargement des modules pgmpy pour éviter les conflits de version
modules_to_reload = [module for module in sys.modules.keys() if module.startswith('pgmpy')]
for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

# Imports principaux
from itertools import combinations
import networkx as nx
import numpy as np
from sklearn.metrics import f1_score

# Imports pgmpy avec gestion d'erreur
try:
    from pgmpy.estimators import PC, HillClimbSearch
    from pgmpy.utils import get_example_model
    print("✅ Imports pgmpy réussis!")
    
    # Test de version
    import pgmpy
    print(f"Version pgmpy: {pgmpy.__version__}")
    print(f"Chemin: {pgmpy.__file__}")
    
except ImportError as e:
    print(f"❌ Erreur import pgmpy: {e}")

# Tentative d'import de GES et SHD
try:
    from pgmpy.estimators import GES
    print("✅ GES disponible!")
except ImportError:
    print("❌ GES non disponible dans cette version")

try:
    from pgmpy.metrics import SHD
    print("✅ SHD disponible!")
except ImportError:
    print("❌ SHD non disponible dans cette version")

✅ Imports pgmpy réussis!
Version pgmpy: 1.0.0
Chemin: /Users/remyplastre/Desktop/IMT ALES 2024-2025/3A/9.2 Advanced_ML/.venv/lib/python3.12/site-packages/pgmpy/__init__.py
✅ GES disponible!
✅ SHD disponible!


In [24]:
# Discrete variable dataset
alarm_model = get_example_model("alarm")
alarm_samples = alarm_model.simulate(int(1e3))
alarm_samples.head()

# Continuous variable dataset
ecoli_model = get_example_model("ecoli70")
ecoli_samples = ecoli_model.simulate(int(1e3))
ecoli_samples.head()

  0%|          | 0/37 [00:00<?, ?it/s]

,b1191,cspG,eutG,fixC,cspA,yecO,yedE,sucA,cchB,yceP,...,dnaK,folK,ycgX,lacZ,nuoM,dnaG,b1583,mopB,yaeM,ftsJ
0,2.912978,1.544210,1.298251,2.440741,-1.017506,1.214050,-1.002049,-3.202744,1.993406,1.325494,...,2.259142,2.995049,3.035647,0.570346,-2.721062,2.427217,1.236712,0.633016,1.399894,1.148165
1,1.771840,2.855767,0.430767,4.363548,-0.041304,2.706498,-1.595230,-1.157455,4.008712,-0.755982,...,0.230434,1.188684,0.348016,1.988618,0.440087,-0.001910,2.800016,-1.035936,5.540998,-0.188362
2,1.216198,3.986883,0.382220,2.578750,1.090017,3.290451,-2.361588,-0.759321,2.900903,-1.176880,...,0.751014,2.182816,1.196445,0.446093,-3.031777,1.294519,1.116771,0.220810,5.299863,0.930846
3,0.700455,2.743655,-0.115917,0.048193,-1.315601,1.809851,-1.557221,-0.997097,0.392430,-0.621334,...,-0.500267,0.525134,-0.302998,1.150075,-1.080963,0.147406,0.981723,-0.897861,5.218121,0.813430
4,1.103168,2.194084,0.679392,1.581695,-1.627669,2.106148,-2.324132,-1.849691,1.717479,0.359200,...,0.381274,1.433114,0.808271,1.881789,-0.752126,0.351351,1.276608,-0.550673,3.962497,0.189979


In [25]:
# Function to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_array(
        estimated_model.to_undirected(), nodelist=nodes, weight=None
    )
    true_adj = nx.to_numpy_array(
        true_model.to_undirected(), nodelist=nodes, weight=None
    )

    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)

## 1. PC algorithm

In [4]:
# Learning the discrete variable alarm model back

est = PC(data=alarm_samples)
estimated_model = est.estimate(ci_test='chi_square', variant="stable", max_cond_vars=4, return_type='dag')
get_f1_score(estimated_model, alarm_model)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'INTUBATION': 'C', 'ANAPHYLAXIS': 'C', 'VENTLUNG': 'C', 'HISTORY': 'C', 'LVEDVOLUME': 'C', 'VENTMACH': 'C', 'TPR': 'C', 'MINVOL': 'C', 'VENTTUBE': 'C', 'HRBP': 'C', 'BP': 'C', 'PAP': 'C', 'PULMEMBOLUS': 'C', 'DISCONNECT': 'C', 'ERRLOWOUTPUT': 'C', 'HYPOVOLEMIA': 'C', 'CVP': 'C', 'HREKG': 'C', 'CO': 'C', 'PRESS': 'C', 'HRSAT': 'C', 'CATECHOL': 'C', 'EXPCO2': 'C', 'PVSAT': 'C', 'ARTCO2': 'C', 'LVFAILURE': 'C', 'KINKEDTUBE': 'C', 'VENTALV': 'C', 'FIO2': 'C', 'SHUNT': 'C', 'PCWP': 'C', 'SAO2': 'C', 'HR': 'C', 'MINVOLSET': 'C', 'STROKEVOLUME': 'C', 'INSUFFANESTH': 'C', 'ERRCAUTER': 'C'}


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:pgmpy:Reached maximum number of allowed conditional variables. Exiting


F1-score for the model skeleton:  0.825


In [5]:
# Learning the continuous variable ecoli model back

est = PC(data=ecoli_samples)
estimated_model = est.estimate(ci_test='pearsonr', variant="orig", max_cond_vars=4, return_type='dag')
get_f1_score(estimated_model, ecoli_model)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'b1191': 'N', 'cspG': 'N', 'eutG': 'N', 'fixC': 'N', 'cspA': 'N', 'yecO': 'N', 'yedE': 'N', 'sucA': 'N', 'cchB': 'N', 'yceP': 'N', 'ygbD': 'N', 'yjbO': 'N', 'yfiA': 'N', 'lpdA': 'N', 'pspB': 'N', 'atpG': 'N', 'dnaJ': 'N', 'flgD': 'N', 'gltA': 'N', 'sucD': 'N', 'tnaA': 'N', 'ygcE': 'N', 'yhdM': 'N', 'ibpB': 'N', 'yfaD': 'N', 'hupB': 'N', 'pspA': 'N', 'asnA': 'N', 'atpD': 'N', 'nmpC': 'N', 'icdA': 'N', 'lacA': 'N', 'yheI': 'N', 'aceB': 'N', 'lacY': 'N', 'b1963': 'N', 'dnaK': 'N', 'folK': 'N', 'ycgX': 'N', 'lacZ': 'N', 'nuoM': 'N', 'dnaG': 'N', 'b1583': 'N', 'mopB': 'N', 'yaeM': 'N', 'ftsJ': 'N'}


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:pgmpy:Reached maximum number of allowed conditional variables. Exiting


F1-score for the model skeleton:  0.640625


## 2. Hill-Climb Search Algorithm

In [6]:
# Learning the discrete variable alarm model back

est = HillClimbSearch(data=alarm_samples)
estimated_model = est.estimate(scoring_method="k2", max_indegree=4, max_iter=int(1e4))
get_f1_score(estimated_model, alarm_model)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'INTUBATION': 'C', 'ANAPHYLAXIS': 'C', 'VENTLUNG': 'C', 'HISTORY': 'C', 'LVEDVOLUME': 'C', 'VENTMACH': 'C', 'TPR': 'C', 'MINVOL': 'C', 'VENTTUBE': 'C', 'HRBP': 'C', 'BP': 'C', 'PAP': 'C', 'PULMEMBOLUS': 'C', 'DISCONNECT': 'C', 'ERRLOWOUTPUT': 'C', 'HYPOVOLEMIA': 'C', 'CVP': 'C', 'HREKG': 'C', 'CO': 'C', 'PRESS': 'C', 'HRSAT': 'C', 'CATECHOL': 'C', 'EXPCO2': 'C', 'PVSAT': 'C', 'ARTCO2': 'C', 'LVFAILURE': 'C', 'KINKEDTUBE': 'C', 'VENTALV': 'C', 'FIO2': 'C', 'SHUNT': 'C', 'PCWP': 'C', 'SAO2': 'C', 'HR': 'C', 'MINVOLSET': 'C', 'STROKEVOLUME': 'C', 'INSUFFANESTH': 'C', 'ERRCAUTER': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'INTUBATION': 'C', 'ANAPHYLAXIS': 'C', 'VENTLUNG': 'C', 'HISTORY': 'C', 'LVEDVOLUME': 'C', 'VENTMACH': 'C', 'TPR': 'C', 'MINVOL': 'C', 'VENTTUBE': 'C', 'HRBP': 'C', 'BP': 'C', 'PAP': 'C', 'PULMEMBOLU

  0%|          | 0/10000 [00:00<?, ?it/s]

F1-score for the model skeleton:  0.7719298245614035


In [7]:
# Learning the continuous variable ecoli model back

est = HillClimbSearch(data=ecoli_samples)
estimated_model = est.estimate(scoring_method="bic-g", max_indegree=4, max_iter=int(1e4))
get_f1_score(estimated_model, ecoli_model)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'b1191': 'N', 'cspG': 'N', 'eutG': 'N', 'fixC': 'N', 'cspA': 'N', 'yecO': 'N', 'yedE': 'N', 'sucA': 'N', 'cchB': 'N', 'yceP': 'N', 'ygbD': 'N', 'yjbO': 'N', 'yfiA': 'N', 'lpdA': 'N', 'pspB': 'N', 'atpG': 'N', 'dnaJ': 'N', 'flgD': 'N', 'gltA': 'N', 'sucD': 'N', 'tnaA': 'N', 'ygcE': 'N', 'yhdM': 'N', 'ibpB': 'N', 'yfaD': 'N', 'hupB': 'N', 'pspA': 'N', 'asnA': 'N', 'atpD': 'N', 'nmpC': 'N', 'icdA': 'N', 'lacA': 'N', 'yheI': 'N', 'aceB': 'N', 'lacY': 'N', 'b1963': 'N', 'dnaK': 'N', 'folK': 'N', 'ycgX': 'N', 'lacZ': 'N', 'nuoM': 'N', 'dnaG': 'N', 'b1583': 'N', 'mopB': 'N', 'yaeM': 'N', 'ftsJ': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'b1191': 'N', 'cspG': 'N', 'eutG': 'N', 'fixC': 'N', 'cspA': 'N', 'yecO': 'N', 'yedE': 'N', 'sucA': 'N', 'cchB': 'N', 'yceP': 'N', 'ygbD': 'N', 'yjbO': 'N', 'yfiA': 'N', 'lpdA': 'N', 'psp

  0%|          | 0/10000 [00:00<?, ?it/s]

F1-score for the model skeleton:  0.7692307692307693


### 3. GES algorithm

In [9]:
# Learning the discrete variable alarm model back

est = GES(data=alarm_samples)
estimated_model = est.estimate(scoring_method="bic-d")
get_f1_score(estimated_model, alarm_model)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'INTUBATION': 'C', 'ANAPHYLAXIS': 'C', 'VENTLUNG': 'C', 'HISTORY': 'C', 'LVEDVOLUME': 'C', 'VENTMACH': 'C', 'TPR': 'C', 'MINVOL': 'C', 'VENTTUBE': 'C', 'HRBP': 'C', 'BP': 'C', 'PAP': 'C', 'PULMEMBOLUS': 'C', 'DISCONNECT': 'C', 'ERRLOWOUTPUT': 'C', 'HYPOVOLEMIA': 'C', 'CVP': 'C', 'HREKG': 'C', 'CO': 'C', 'PRESS': 'C', 'HRSAT': 'C', 'CATECHOL': 'C', 'EXPCO2': 'C', 'PVSAT': 'C', 'ARTCO2': 'C', 'LVFAILURE': 'C', 'KINKEDTUBE': 'C', 'VENTALV': 'C', 'FIO2': 'C', 'SHUNT': 'C', 'PCWP': 'C', 'SAO2': 'C', 'HR': 'C', 'MINVOLSET': 'C', 'STROKEVOLUME': 'C', 'INSUFFANESTH': 'C', 'ERRCAUTER': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'INTUBATION': 'C', 'ANAPHYLAXIS': 'C', 'VENTLUNG': 'C', 'HISTORY': 'C', 'LVEDVOLUME': 'C', 'VENTMACH': 'C', 'TPR': 'C', 'MINVOL': 'C', 'VENTTUBE': 'C', 'HRBP': 'C', 'BP': 'C', 'PAP': 'C', 'PULMEMBOLU

F1-score for the model skeleton:  0.8444444444444444


In [10]:
# Learning the continuous variable ecoli model back

est = GES(data=ecoli_samples)
estimated_model = est.estimate(scoring_method="bic-g")
get_f1_score(estimated_model, ecoli_model)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'b1191': 'N', 'cspG': 'N', 'eutG': 'N', 'fixC': 'N', 'cspA': 'N', 'yecO': 'N', 'yedE': 'N', 'sucA': 'N', 'cchB': 'N', 'yceP': 'N', 'ygbD': 'N', 'yjbO': 'N', 'yfiA': 'N', 'lpdA': 'N', 'pspB': 'N', 'atpG': 'N', 'dnaJ': 'N', 'flgD': 'N', 'gltA': 'N', 'sucD': 'N', 'tnaA': 'N', 'ygcE': 'N', 'yhdM': 'N', 'ibpB': 'N', 'yfaD': 'N', 'hupB': 'N', 'pspA': 'N', 'asnA': 'N', 'atpD': 'N', 'nmpC': 'N', 'icdA': 'N', 'lacA': 'N', 'yheI': 'N', 'aceB': 'N', 'lacY': 'N', 'b1963': 'N', 'dnaK': 'N', 'folK': 'N', 'ycgX': 'N', 'lacZ': 'N', 'nuoM': 'N', 'dnaG': 'N', 'b1583': 'N', 'mopB': 'N', 'yaeM': 'N', 'ftsJ': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'b1191': 'N', 'cspG': 'N', 'eutG': 'N', 'fixC': 'N', 'cspA': 'N', 'yecO': 'N', 'yedE': 'N', 'sucA': 'N', 'cchB': 'N', 'yceP': 'N', 'ygbD': 'N', 'yjbO': 'N', 'yfiA': 'N', 'lpdA': 'N', 'psp

F1-score for the model skeleton:  0.8461538461538461


## 4. Expert In Loop Algorithm

Please refer to the following blogpost for more details: https://medium.com/gopenai/llms-for-causal-discovery-745e2cba0b59